In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from pycaret.regression import setup, compare_models
import logging
import transformers
from sklearn.metrics import accuracy_score
import optuna
from sklearn.model_selection import KFold
from sklearn.experimental import enable_iterative_imputer
import itertools



In [2]:
trainUsers=pd.read_csv("users_train.csv")
trainUsers.drop("ID",axis=1,inplace=True)
testUsers=pd.read_csv("users_test.csv")
testUsers.drop("ID",axis=1,inplace=True)
trainFeatures=pd.read_csv("user_features_train.csv")
testFeatures=pd.read_csv("user_features_test.csv")
target=pd.read_csv("targets_train.csv")
train=[trainUsers,trainFeatures,target]
test=[testUsers,testFeatures]
train=pd.concat(train,axis=1)
test=pd.concat(test,axis=1)


ID=test["ID"]

In [3]:
def process_datetime_features(df):
    # `first_open_date` ve `first_open_timestamp` sütunlarının tarih formatında olduğundan emin olun
    df['first_open_date'] = pd.to_datetime(df['first_open_date'])
    df['first_open_timestamp'] = pd.to_datetime(df['first_open_timestamp'])
    df['local_first_open_timestamp'] = pd.to_datetime(df['local_first_open_timestamp'])

    # Tarih bileşenlerini ayırın
    df['open_year'] = df['first_open_date'].dt.year
    df['open_month'] = df['first_open_date'].dt.month
    df['open_day'] = df['first_open_date'].dt.day

    # `microsecond` değerlerine erişim
    df["micro_first_open_times"] = df['first_open_timestamp'].dt.microsecond
    df["local_micro_first_open_times"] = df['local_first_open_timestamp'].dt.microsecond

    # Orijinal datetime sütunlarını kaldırın
    df = df.drop(columns=['first_open_date', 'first_open_timestamp', 'local_first_open_timestamp'],inplace=True)

    return df
process_datetime_features(train)
process_datetime_features(test)
process_datetime_features(testUsers)

In [4]:
trainUserscopy=trainUsers.iloc[:10000]
testUserscopy=testUsers.iloc[:10000]
trainFeaturescopy=trainFeatures.iloc[:10000]
testFeaturescopy=testFeatures.iloc[:10000]


In [5]:
train.shape,test.shape

((878594, 90), (585730, 88))

In [6]:
train.drop("ID",inplace=True,axis=1)
test.drop("ID",inplace=True,axis=1)

test.shape,train.shape


((585730, 87), (878594, 88))

In [7]:
#çalıştır
def check_data(dataframe):
    print(20*"-" + "Information".center(20) + 20*"-")
    print(dataframe.info())
    print("\n" + 20*"-" + "The First 5 Data".center(20) + 20*"-")
    print(dataframe.head(20))
    print(20*"-" + "Data Shape".center(20) + 20*"-")
    print(dataframe.shape)

check_data(train)



--------------------    Information     --------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878594 entries, 0 to 878593
Data columns (total 88 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   country                       878512 non-null  object 
 1   platform                      878594 non-null  object 
 2   device_category               878594 non-null  object 
 3   device_brand                  872754 non-null  object 
 4   device_model                  878594 non-null  object 
 5   has_ios_att_permission        878594 non-null  bool   
 6   ad_network                    568124 non-null  object 
 7   first_prediction              852859 non-null  float64
 8   RetentionD0                   878594 non-null  bool   
 9   RetentionD1                   878594 non-null  bool   
 10  RetentionD2                   878594 non-null  bool   
 11  RetentionD3                   878594 non-nu

In [8]:

# IAPRevenue için benzer sütunlar oluşturma
train["IAPRevenueTotal2"] = train["IAPRevenueD0"] + train["IAPRevenueD1"]
train["IAPRevenueTotal4"] = train["IAPRevenueD2"] + train["IAPRevenueD3"]
train["IAPRevenueTotal6"] = train["IAPRevenueD4"] + train["IAPRevenueD5"]
train["IAPRevenueTotal8"] = train["IAPRevenueD6"] + train["IAPRevenueD7"]
train["IAPRevenueTotal10"] = train["IAPRevenueD8"] + train["IAPRevenueD9"]
train["IAPRevenueTotal12"] = train["IAPRevenueD10"] + train["IAPRevenueD11"]
train["IAPRevenueTotal14"] = train["IAPRevenueD12"] + train["IAPRevenueD13"]
train["IAPRevenueTotal16"] = train["IAPRevenueD14"] + train["IAPRevenueD15"]


# LevelAdvancedCountTotal
train["LevelAdvancedCountTotal2"] = train["LevelAdvancedCountD0"] + train["LevelAdvancedCountD1"]
train["LevelAdvancedCountTotal4"] = train["LevelAdvancedCountD2"] + train["LevelAdvancedCountD3"]
train["LevelAdvancedCountTotal6"] = train["LevelAdvancedCountD4"] + train["LevelAdvancedCountD5"]
train["LevelAdvancedCountTotal8"] = train["LevelAdvancedCountD6"] + train["LevelAdvancedCountD7"]
train["LevelAdvancedCountTotal10"] = train["LevelAdvancedCountD8"] + train["LevelAdvancedCountD9"]
train["LevelAdvancedCountTotal12"] = train["LevelAdvancedCountD10"] + train["LevelAdvancedCountD11"]
train["LevelAdvancedCountTotal14"] = train["LevelAdvancedCountD12"] + train["LevelAdvancedCountD13"]
train["LevelAdvancedCountTotal16"] = train["LevelAdvancedCountD14"] + train["LevelAdvancedCountD15"]

# Level Duration
train["LevelDurationTotal2"] = train["Level_1_Duration"] + train["Level_2_Duration"]
train["LevelDurationTotal4"] = train["Level_3_Duration"] + train["Level_4_Duration"]
train["LevelDurationTotal6"] = train["Level_5_Duration"] + train["Level_6_Duration"]
train["LevelDurationTotal8"] = train["Level_7_Duration"] + train["Level_8_Duration"]
train["LevelDurationTotal10"] = train["Level_9_Duration"] + train["Level_10_Duration"]

# AdRevenue
train["AdRevenueTotal2"] = train["AdRevenueD0"] + train["AdRevenueD1"]
train["AdRevenueTotal4"] = train["AdRevenueD2"] + train["AdRevenueD3"]
train["AdRevenueTotal6"] = train["AdRevenueD4"] + train["AdRevenueD5"]
train["AdRevenueTotal8"] = train["AdRevenueD6"] + train["AdRevenueD7"]
train["AdRevenueTotal10"] = train["AdRevenueD8"] + train["AdRevenueD9"]
train["AdRevenueTotal12"] = train["AdRevenueD10"] + train["AdRevenueD11"]
train["AdRevenueTotal14"] = train["AdRevenueD12"] + train["AdRevenueD13"]
train["AdRevenueTotal16"] = train["AdRevenueD14"] + train["AdRevenueD15"]


# Retention için benzer sütunlar oluşturma (train seti)
train["RetentionTotal2"] = train["RetentionD0"] + train["RetentionD1"]
train["RetentionTotal4"] = train["RetentionD2"] + train["RetentionD3"]
train["RetentionTotal6"] = train["RetentionD4"] + train["RetentionD5"]
train["RetentionTotal8"] = train["RetentionD6"] + train["RetentionD7"]
train["RetentionTotal10"] = train["RetentionD8"] + train["RetentionD9"]
train["RetentionTotal12"] = train["RetentionD10"] + train["RetentionD11"]
train["RetentionTotal14"] = train["RetentionD12"] + train["RetentionD13"]
train["RetentionTotal16"] = train["RetentionD14"] + train["RetentionD15"]



# LevelAdvancedCountTotal
test["LevelAdvancedCountTotal2"] = test["LevelAdvancedCountD0"] + test["LevelAdvancedCountD1"]
test["LevelAdvancedCountTotal4"] = test["LevelAdvancedCountD2"] + test["LevelAdvancedCountD3"]
test["LevelAdvancedCountTotal6"] = test["LevelAdvancedCountD4"] + test["LevelAdvancedCountD5"]
test["LevelAdvancedCountTotal8"] = test["LevelAdvancedCountD6"] + test["LevelAdvancedCountD7"]
test["LevelAdvancedCountTotal10"] = test["LevelAdvancedCountD8"] + test["LevelAdvancedCountD9"]
test["LevelAdvancedCountTotal12"] = test["LevelAdvancedCountD10"] + test["LevelAdvancedCountD11"]
test["LevelAdvancedCountTotal14"] = test["LevelAdvancedCountD12"] + test["LevelAdvancedCountD13"]
test["LevelAdvancedCountTotal16"] = test["LevelAdvancedCountD14"] + test["LevelAdvancedCountD15"]

# Level Duration
test["LevelDurationTotal2"] = test["Level_1_Duration"] + test["Level_2_Duration"]
test["LevelDurationTotal4"] = test["Level_3_Duration"] + test["Level_4_Duration"]
test["LevelDurationTotal6"] = test["Level_5_Duration"] + test["Level_6_Duration"]
test["LevelDurationTotal8"] = test["Level_7_Duration"] + test["Level_8_Duration"]
test["LevelDurationTotal10"] = test["Level_9_Duration"] + test["Level_10_Duration"]

# AdRevenue
test["AdRevenueTotal2"] = test["AdRevenueD0"] + test["AdRevenueD1"]
test["AdRevenueTotal4"] = test["AdRevenueD2"] + test["AdRevenueD3"]
test["AdRevenueTotal6"] = test["AdRevenueD4"] + test["AdRevenueD5"]
test["AdRevenueTotal8"] = test["AdRevenueD6"] + test["AdRevenueD7"]
test["AdRevenueTotal10"] = test["AdRevenueD8"] + test["AdRevenueD9"]
test["AdRevenueTotal12"] = test["AdRevenueD10"] + test["AdRevenueD11"]
test["AdRevenueTotal14"] = test["AdRevenueD12"] + test["AdRevenueD13"]
test["AdRevenueTotal16"] = test["AdRevenueD14"] + test["AdRevenueD15"]


# IAPRevenue için benzer sütunlar oluşturma (test seti)
test["IAPRevenueTotal2"] = test["IAPRevenueD0"] + test["IAPRevenueD1"]
test["IAPRevenueTotal4"] = test["IAPRevenueD2"] + test["IAPRevenueD3"]
test["IAPRevenueTotal6"] = test["IAPRevenueD4"] + test["IAPRevenueD5"]
test["IAPRevenueTotal8"] = test["IAPRevenueD6"] + test["IAPRevenueD7"]
test["IAPRevenueTotal10"] = test["IAPRevenueD8"] + test["IAPRevenueD9"]
test["IAPRevenueTotal12"] = test["IAPRevenueD10"] + test["IAPRevenueD11"]
test["IAPRevenueTotal14"] = test["IAPRevenueD12"] + test["IAPRevenueD13"]
test["IAPRevenueTotal16"] = test["IAPRevenueD14"] + test["IAPRevenueD15"]


# Retention için benzer sütunlar oluşturma (test seti)
test["RetentionTotal2"] = test["RetentionD0"] + test["RetentionD1"]
test["RetentionTotal4"] = test["RetentionD2"] + test["RetentionD3"]
test["RetentionTotal6"] = test["RetentionD4"] + test["RetentionD5"]
test["RetentionTotal8"] = test["RetentionD6"] + test["RetentionD7"]
test["RetentionTotal10"] = test["RetentionD8"] + test["RetentionD9"]
test["RetentionTotal12"] = test["RetentionD10"] + test["RetentionD11"]
test["RetentionTotal14"] = test["RetentionD12"] + test["RetentionD13"]
test["RetentionTotal16"] = test["RetentionD14"] + test["RetentionD15"]



In [9]:

#LevelAdvancedCountTotal
train["LevelAdvancedCountTotal5"] = (
        train["LevelAdvancedCountD0"] + train["LevelAdvancedCountD1"] + train["LevelAdvancedCountD2"] + train["LevelAdvancedCountD3"]+ train["LevelAdvancedCountD4"]
)

train["LevelAdvancedCountTotal10"] = (
        train["LevelAdvancedCountD5"] + train["LevelAdvancedCountD6"] + train["LevelAdvancedCountD7"] + train["LevelAdvancedCountD8"]+ train["LevelAdvancedCountD9"]
)

train["LevelAdvancedCountTotal15"] = (
        train["LevelAdvancedCountD10"] + train["LevelAdvancedCountD11"] + train["LevelAdvancedCountD12"] + train["LevelAdvancedCountD13"]+ train["LevelAdvancedCountD14"]+train["LevelAdvancedCountD15"]
)

liste=["LevelDurationTotal5","LevelDurationTotal10","LevelAdvancedCountTotal5","LevelAdvancedCountTotal10","LevelAdvancedCountTotal15",
       "RetentionTotal5","RetentionTotal10","RetentionTotal15","IAPRevenueTotal5","IAPRevenueTotal15"]
#Level Duration 
train["LevelDurationTotal5"] = (
        train["Level_1_Duration"] + train["Level_2_Duration"] + train["Level_3_Duration"] +
        train["Level_4_Duration"] + train["Level_5_Duration"]
)
train["LevelDurationTotal10"] = (
        train["Level_6_Duration"] + train["Level_7_Duration"] + train["Level_8_Duration"] +
        train["Level_9_Duration"] + train["Level_10_Duration"]
)





#AdRevenue
train["AdRevenueTotal5"] = (
        train["AdRevenueD0"] + train["AdRevenueD1"] + train["AdRevenueD2"] +
        train["AdRevenueD3"] + train["AdRevenueD4"]
)
train["AdRevenueTotal10"] = (
        train["AdRevenueD5"] + train["AdRevenueD6"] + train["AdRevenueD7"] +
        train["AdRevenueD8"] + train["AdRevenueD9"]
)
train["AdRevenueTotal15"] = (
        train["AdRevenueD10"] + train["AdRevenueD11"] + train["AdRevenueD12"] +
        train["AdRevenueD13"] + train["AdRevenueD14"]
)


# Retention Total
train["RetentionTotal5"] = (
        train["RetentionD0"] + train["RetentionD1"] + train["RetentionD2"] +
        train["RetentionD3"] + train["RetentionD4"]
)
train["RetentionTotal10"] = (
        train["RetentionD5"] + train["RetentionD6"] + train["RetentionD7"] +
        train["RetentionD8"] + train["RetentionD9"]
)
train["RetentionTotal15"] = (
        train["RetentionD10"] + train["RetentionD11"] + train["RetentionD12"] +
        train["RetentionD13"] + train["RetentionD14"]
)

# IAPRevenue Total
train["IAPRevenueTotal5"] = (
        train["IAPRevenueD0"] + train["IAPRevenueD1"] + train["IAPRevenueD2"] +
        train["IAPRevenueD3"] + train["IAPRevenueD4"]
)
train["IAPRevenueTotal10"] = (
        train["IAPRevenueD5"] + train["IAPRevenueD6"] + train["IAPRevenueD7"] +
        train["IAPRevenueD8"] + train["IAPRevenueD9"]
)
train["IAPRevenueTotal15"] = (
        train["IAPRevenueD10"] + train["IAPRevenueD11"] + train["IAPRevenueD12"] +
        train["IAPRevenueD13"] + train["IAPRevenueD14"]
)











#test






#LevelAdvancedCountTotal
test["LevelAdvancedCountTotal5"] = (
        test["LevelAdvancedCountD0"] + test["LevelAdvancedCountD1"] + test["LevelAdvancedCountD2"] + test["LevelAdvancedCountD3"]+ test["LevelAdvancedCountD4"]
)

test["LevelAdvancedCountTotal10"] = (
        test["LevelAdvancedCountD5"] + test["LevelAdvancedCountD6"] + test["LevelAdvancedCountD7"] + test["LevelAdvancedCountD8"]+ test["LevelAdvancedCountD9"]
)

test["LevelAdvancedCountTotal15"] = (
        test["LevelAdvancedCountD10"] + test["LevelAdvancedCountD11"] + test["LevelAdvancedCountD12"] + test["LevelAdvancedCountD13"]+ test["LevelAdvancedCountD14"]+test["LevelAdvancedCountD15"]
)


#Level Duration 
test["LevelDurationTotal5"] = (
        test["Level_1_Duration"] + test["Level_2_Duration"] + test["Level_3_Duration"] +
        test["Level_4_Duration"] + test["Level_5_Duration"]
)
test["LevelDurationTotal10"] = (
        test["Level_6_Duration"] + test["Level_7_Duration"] + test["Level_8_Duration"] +
        test["Level_9_Duration"] + test["Level_10_Duration"]
)





#AdRevenue
test["AdRevenueTotal5"] = (
        test["AdRevenueD0"] + test["AdRevenueD1"] + test["AdRevenueD2"] +
        test["AdRevenueD3"] + test["AdRevenueD4"]
)
test["AdRevenueTotal10"] = (
        test["AdRevenueD5"] + test["AdRevenueD6"] + test["AdRevenueD7"] +
        test["AdRevenueD8"] + test["AdRevenueD9"]
)
test["AdRevenueTotal15"] = (
        test["AdRevenueD10"] + test["AdRevenueD11"] + test["AdRevenueD12"] +
        test["AdRevenueD13"] + test["AdRevenueD14"]
)

# Aynı işlemleri test setine de uyguluyoruz
test["RetentionTotal5"] = (
        test["RetentionD0"] + test["RetentionD1"] + test["RetentionD2"] +
        test["RetentionD3"] + test["RetentionD4"]
)
test["RetentionTotal10"] = (
        test["RetentionD5"] + test["RetentionD6"] + test["RetentionD7"] +
        test["RetentionD8"] + test["RetentionD9"]
)
test["RetentionTotal15"] = (
        test["RetentionD10"] + test["RetentionD11"] + test["RetentionD12"] +
        test["RetentionD13"] + test["RetentionD14"]
)


# Aynı işlemleri test setine de uyguluyoruz
test["IAPRevenueTotal5"] = (
        test["IAPRevenueD0"] + test["IAPRevenueD1"] + test["IAPRevenueD2"] +
        test["IAPRevenueD3"] + test["IAPRevenueD4"]
)
test["IAPRevenueTotal10"] = (
        test["IAPRevenueD5"] + test["IAPRevenueD6"] + test["IAPRevenueD7"] +
        test["IAPRevenueD8"] + test["IAPRevenueD9"]
)
test["IAPRevenueTotal15"] = (
        test["IAPRevenueD10"] + test["IAPRevenueD11"] + test["IAPRevenueD12"] +
        test["IAPRevenueD13"] + test["IAPRevenueD14"]
)


train['ad_network'] = train['ad_network'].fillna('organic')
test['ad_network'] = test['ad_network'].fillna('organic')




In [10]:






train["first_prediction"]=train["first_prediction"].interpolate(method="linear")
test["first_prediction"]=test["first_prediction"].interpolate(method="linear")





#BAŞARISIZ
#network_revenue_avg = test.groupby('ad_network')['Total_Revenue'].mean().reset_index()
#network_revenue_avg.columns = ['ad_network', 'AdNetwork_Revenue_Avg']

# Kullanıcı verisine ekleyelim
#test = test.merge(network_revenue_avg, on='ad_network', how='left')
#train["first_open_timestamp"]=np.log1p(train["first_open_timestamp"])
#test["first_open_timestamp"]=np.log1p(test["first_open_timestamp"])

#test["local_first_open_timestamp"]=np.log1p(test["local_first_open_timestamp"])
#train["local_first_open_timestamp"]=np.log1p(train["local_first_open_timestamp"])


# İlk IAP satın almasının yapıldığı günü bulmak için bir fonksiyon yazalım
#def days_to_first_iap(row):
#    for i in range(1, 16):
#        if row[f'IAPRevenueD{i}'] > 0:
#            return i
#    return None

# Kullanıcı başına kaçıncı gün ilk satın almanın yapıldığını hesaplayalım
#train['Days_to_First_IAP'] = train.apply(days_to_first_iap, axis=1)
#test["Days_to_First_IAP"]=train.apply(days_to_first_iap,axis=1)




####################
train['Total_Revenue'] = train.filter(like='AdRevenueD').sum(axis=1) + train.filter(like='IAPRevenueD').sum(axis=1)
test['Total_Revenue'] = test.filter(like='AdRevenueD').sum(axis=1) + test.filter(like='IAPRevenueD').sum(axis=1)
######################

#############################
network_revenue_avg = train.groupby('ad_network')['Total_Revenue'].mean().reset_index()
network_revenue_avg.columns = ['ad_network', 'AdNetwork_Revenue_Avg']
train = train.merge(network_revenue_avg, on='ad_network', how='left')


network_revenue_avg = test.groupby('ad_network')['Total_Revenue'].mean().reset_index()
network_revenue_avg.columns = ['ad_network', 'AdNetwork_Revenue_Avg']
test = test.merge(network_revenue_avg, on='ad_network', how='left')
###############################





################################
# Device Brand'e göre ortalama gelir
device_brand_revenue_avg = train.groupby('device_brand')['Total_Revenue'].mean().reset_index()
device_brand_revenue_avg.columns = ['device_brand', 'Device_Brand_Revenue_Avg']
train = train.merge(device_brand_revenue_avg, on='device_brand', how='left')

device_brand_revenue_avg = test.groupby('device_brand')['Total_Revenue'].mean().reset_index()
device_brand_revenue_avg.columns = ['device_brand', 'Device_Brand_Revenue_Avg']
test = test.merge(device_brand_revenue_avg, on='device_brand', how='left')
###############################


#############################
# Ülke, cihaz ve platform kombinasyonuna göre ortalama gelir
country_device_platform_revenue_avg = train.groupby(['country', 'device_category', 'platform'])['Total_Revenue'].mean().reset_index()
country_device_platform_revenue_avg.columns = ['country', 'device_category', 'platform', 'Country_Device_Platform_Revenue_Avg']
train = train.merge(country_device_platform_revenue_avg, on=['country', 'device_category', 'platform'], how='left')


country_device_platform_revenue_avg = test.groupby(['country', 'device_category', 'platform'])['Total_Revenue'].mean().reset_index()
country_device_platform_revenue_avg.columns = ['country', 'device_category', 'platform', 'Country_Device_Platform_Revenue_Avg']
test = test.merge(country_device_platform_revenue_avg, on=['country', 'device_category', 'platform'], how='left')
###############################

#train["first_open_date"] = pd.to_datetime(train["first_open_date"])
#train["first_open_timestamp"] = pd.to_timedelta(train["first_open_timestamp"])
#train["local_first_open_timestamp"] = pd.to_timedelta(train["local_first_open_timestamp"])

#test["first_open_date"] = pd.to_datetime(test["first_open_date"])
#test["first_open_timestamp"] = pd.to_timedelta(test["first_open_timestamp"])
#test["local_first_open_timestamp"] = pd.to_timedelta(test["local_first_open_timestamp"])



train["score4"]=(train["Total_Revenue"]+train["first_prediction"])**2



In [11]:
#score 1 BAŞARILI
train["score1"]=(train["LevelAdvancedCountD10"]+train["LevelAdvancedCountD11"]+train["LevelAdvancedCountD12"]+train["LevelAdvancedCountD13"])*train["Total_Revenue"]
test["score1"]=(test["LevelAdvancedCountD10"]+test["LevelAdvancedCountD11"]+test["LevelAdvancedCountD12"]+test["LevelAdvancedCountD13"])*test["Total_Revenue"]

#score 2 BAŞARILI
train["score2"]=(train["Country_Device_Platform_Revenue_Avg"]+train["first_prediction"])**2
test["score2"]=(test["Country_Device_Platform_Revenue_Avg"]+test["first_prediction"])**2

#Score3 BAŞARISIZ
#train["score3"]=(train["AdRevenueD13"]+train["AdRevenueD1"]+train["AdRevenueD12"]+train["AdRevenueD3"]+train["AdRevenueD4"])**2

#test["score3"]=(test["AdRevenueD13"]+test["AdRevenueD1"]+test["AdRevenueD12"]+test["AdRevenueD3"]+test["AdRevenueD4"])**2

#Score 3 BAŞARILI
train["score3"]=(train["LevelAdvancedCountTotal15"]+train["LevelAdvancedCountTotal10"])**2
test["score3"]=(test["LevelAdvancedCountTotal15"]+test["LevelAdvancedCountTotal10"])**2



In [12]:

# 1. En uzun aktif kalma süresi (streak)
def calculate_longest_streak(df):
    retention_columns = [
        'RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5',
        'RetentionD6', 'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10',
        'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14', 'RetentionD15'
    ]
    df['longest_streak'] = df[retention_columns].apply(
        lambda row: max([len(list(g)) for k, g in itertools.groupby(row) if k == True], default=0), axis=1
    )
    return df

# 2. En uzun streak süresince getirilen toplam gelir
def calculate_streak_revenue(df):
    ad_revenue_columns = ['AdRevenueD' + str(i) for i in range(1, 16)]
    iap_revenue_columns = ['IAPRevenueD' + str(i) for i in range(1, 16)]
    retention_columns = ['RetentionD' + str(i) for i in range(1, 16)]

    df['streak_revenue'] = df.apply(
        lambda row: sum(row[ad_revenue_columns[i]] + row[iap_revenue_columns[i]] for i in range(15) if row[retention_columns[i]]), axis=1
    )
    return df

# 3. İlk tahmin edilen değer ile level tamamlama süresi arasındaki fark
def calculate_prediction_vs_duration(df):
    df['avg_level_duration'] = df[['Level_' + str(i) + '_Duration' for i in range(1, 11)]].mean(axis=1, skipna=True)
    df['first_prediction_vs_duration'] = df['first_prediction'] - df['avg_level_duration']
    return df

# 4. Reklamların sıklığının kullanıcı geliri üzerindeki etkisi
def calculate_ad_revenue_ratio(df):
    ad_revenue_columns = ['AdRevenueD' + str(i) for i in range(1, 16)]
    iap_revenue_columns = ['IAPRevenueD' + str(i) for i in range(1, 16)]

    df['ad_revenue_ratio'] = df[ad_revenue_columns].sum(axis=1) / (
            df[ad_revenue_columns].sum(axis=1) + df[iap_revenue_columns].sum(axis=1)
    ).replace(0, 1)  # Bölme hatası olmasın diye 0'ı 1 ile değiştiriyoruz
    return df

# 5. Reklam izlemeyen kullanıcıların durumu
def calculate_no_ad_revenue_days(df):
    ad_revenue_columns = ['AdRevenueD' + str(i) for i in range(1, 16)]

    df['no_ad_revenue_days'] = df[ad_revenue_columns].apply(lambda row: sum([1 for x in row if x == 0]), axis=1)
    df['avg_revenue_no_ads'] = df[['IAPRevenueD' + str(i) for i in range(1, 16)]].mean(axis=1)
    return df

# 6. İlk satın alma işlemi gerçekleşene kadar geçen gün
def calculate_first_purchase_day(df):
    df['first_purchase_day'] = df[['IAPRevenueD' + str(i) for i in range(1, 16)]].apply(
        lambda row: next((i+1 for i, x in enumerate(row) if x > 0), 0), axis=1
    )
    return df

# 7. Satın alma yapmama süresi
def calculate_days_without_purchase(df):
    df['days_without_purchase'] = df[['IAPRevenueD' + str(i) for i in range(1, 16)]].apply(
        lambda row: sum([1 for x in row if x == 0]), axis=1
    )
    return df

# 8. İlk 15 günlük ortalama gelir
def calculate_avg_15_days_revenue(df):
    df['avg_15_days_revenue'] = (
                                        df[['AdRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1) +
                                        df[['IAPRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1)
                                ) / 15
    return df

# 9. İlk 15 günlük ortalama oyun süresi
def calculate_avg_15_days_playtime(df):
    df['avg_15_days_playtime'] = df[['Level_' + str(i) + '_Duration' for i in range(1, 16)]].mean(axis=1, skipna=True)
    return df

# 10. İlk 15 günlük reklam geliri oranı
def calculate_ad_revenue_ratio_15_days(df):
    df['ad_revenue_ratio_15_days'] = df[['AdRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1) / (
            df[['AdRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1) + df[['IAPRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1)
    ).replace(0, 1)  # 0 olan değerleri 1 ile değiştiriyoruz
    return df

# 11. İlk 15 günlük satın alma geliri oranı
def calculate_iap_revenue_ratio_15_days(df):
    df['iap_revenue_ratio_15_days'] = df[['IAPRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1) / (
            df[['AdRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1) + df[['IAPRevenueD' + str(i) for i in range(1, 16)]].sum(axis=1)
    ).replace(0, 1)
    return df

# 12. İlk 15 günlük gelir tahmini hatası
def calculate_revenue_prediction_error(df):
    df['revenue_prediction_error'] = df['first_prediction'] - df['TARGET']
    return df

# 13. En çok zaman geçirilen günlerdeki gelir
def calculate_most_played_day_revenue(df):
    df['most_played_day_revenue'] = df[['Level_' + str(i) + '_Duration' for i in range(1, 16)]].idxmax(axis=1).apply(
        lambda x: int(x.split('_')[1])
    ).apply(
        lambda i: df['AdRevenueD' + str(i)] + df['IAPRevenueD' + str(i)]
    )
    return df

# 14. İlk 15 günde gelinen ortalama seviye
def calculate_avg_level_completed(df):
    df['avg_level_completed'] = df[['LevelAdvancedCountD' + str(i) for i in range(1, 16)]].mean(axis=1, skipna=True)
    return df

# 15. En düşük gelir getiren seviye
def calculate_min_revenue_level(df):
    df['min_revenue_level'] = df[['LevelAdvancedCountD' + str(i) for i in range(1, 16)] + ['AdRevenueD' + str(i) for i in range(1, 16)] + ['IAPRevenueD' + str(i) for i in range(1, 16)]].apply(
        lambda row: min(row), axis=1
    )
    return df

# 16. En yüksek gelir getiren seviye
def calculate_max_revenue_level(df):
    df['max_revenue_level'] = df[['LevelAdvancedCountD' + str(i) for i in range(1, 16)] + ['AdRevenueD' + str(i) for i in range(1, 16)] + ['IAPRevenueD' + str(i) for i in range(1, 16)]].apply(
        lambda row: max(row), axis=1
    )
    return df



,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network,first_prediction,RetentionD0,RetentionD1,...,IAPRevenueTotal5,IAPRevenueTotal15,Total_Revenue,AdNetwork_Revenue_Avg,Device_Brand_Revenue_Avg,Country_Device_Platform_Revenue_Avg,score1,score2,score3,ad_revenue_ratio
0,Argentina,Android,mobile,Motorola,Moto G32,False,applovin_int,1.444805,True,True,...,0.0,0.0,0.012716,0.397935,0.137990,0.041591,0.000000,2.209374,0,1.0
1,Mexico,Android,mobile,OnePlus,Nord N20 SE,False,applovin_int,9.147972,True,False,...,0.0,0.0,0.000000,0.397935,0.198309,0.110217,0.000000,85.714067,0,0.0
2,France,Android,mobile,Motorola,moto g13,False,applovin_int,40.731158,True,False,...,0.0,0.0,0.000000,0.397935,0.137990,0.426031,0.000000,1693.914200,0,0.0
3,Brazil,Android,mobile,Samsung,Galaxy A03,False,applovin_int,4.967959,True,False,...,0.0,0.0,0.269068,0.397935,0.348405,0.129514,0.000000,25.984224,361,1.0
4,Peru,Android,mobile,Xiaomi,Redmi 13C,False,applovin_int,2.445842,True,False,...,0.0,0.0,0.004202,0.397935,0.116755,0.048148,0.000000,6.219986,81,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585725,Argentina,Android,mobile,Motorola,Moto G52,False,applovin_int,3.184875,True,False,...,0.0,0.0,0.000000,0.397935,0.137990,0.041591,0.000000,10.410081,0,0.0
585726,United States,iOS,mobile,Apple,iPhone 14 Plus,True,applovin_int,234.374702,True,False,...,0.0,0.0,3.500052,0.397935,0.473222,1.226871,0.000000,55508.101004,0,0.0
585727,United States,iOS,mobile,Apple,iPhone 11,False,applovin_int,101.114631,True,True,...,0.0,0.0,1.359659,0.397935,0.473222,1.226871,12.236929,10473.782890,81,1.0
585728,Argentina,Android,mobile,Samsung,Galaxy A22 4G,False,applovin_int,1.341221,True,True,...,0.0,0.0,0.007589,0.397935,0.348405,0.041591,0.000000,1.912169,0,1.0


train['ad_revenue_per_retention_day'] = (train['total_ad_revenue'] / train['total_retention_days'].replace(0, 1))
train['level_completion_rate'] = (train['total_levels_completed'] / train['total_retention_days'].replace(0, 1))
adrevenue_columns = [f'AdRevenueD{i}' for i in range(1, 16)]
train['total_ad_revenue'] = train[adrevenue_columns].sum(axis=1)

retention_columns = [f'RetentionD{i}' for i in range(1, 16)]
train['total_retention_days'] = train[retention_columns].sum(axis=1)
LevelAdvancedCountD_columns = [f'LevelAdvancedCountD{i}' for i in range(1, 16)]
train['total_levels_completed'] = train[LevelAdvancedCountD_columns].sum(axis=1)
retention_days = ['RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6', 'RetentionD7']
train['early_adopter_flag'] = train[retention_days].sum(axis=1) >= 4





train['ad_revenue_per_retention_day'] = (train['total_ad_revenue'] / train['total_retention_days'].replace(0, 1))
train['level_completion_rate'] = (train['total_levels_completed'] / train['total_retention_days'].replace(0, 1))
adrevenue_columns = [f'AdRevenueD{i}' for i in range(1, 16)]
train['total_ad_revenue'] = train[adrevenue_columns].sum(axis=1)

retention_columns = [f'RetentionD{i}' for i in range(1, 16)]
train['total_retention_days'] = train[retention_columns].sum(axis=1)
LevelAdvancedCountD_columns = [f'LevelAdvancedCountD{i}' for i in range(1, 16)]
train['total_levels_completed'] = train[LevelAdvancedCountD_columns].sum(axis=1)
retention_days = ['RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6', 'RetentionD7']
train['early_adopter_flag'] = train[retention_days].sum(axis=1) >= 4





test['ad_revenue_per_retention_day'] = (test['total_ad_revenue'] / test['total_retention_days'].replace(0, 1))
test['level_completion_rate'] = (test['total_levels_completed'] / test['total_retention_days'].replace(0, 1))
adrevenue_columns = [f'AdRevenueD{i}' for i in range(1, 16)]
test['total_ad_revenue'] = test[adrevenue_columns].sum(axis=1)

retention_columns = [f'RetentionD{i}' for i in range(1, 16)]
test['total_retention_days'] = test[retention_columns].sum(axis=1)
LevelAdvancedCountD_columns = [f'LevelAdvancedCountD{i}' for i in range(1, 16)]
test['total_levels_completed'] = test[LevelAdvancedCountD_columns].sum(axis=1)
retention_days = ['RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6', 'RetentionD7']
test['early_adopter_flag'] = test[retention_days].sum(axis=1) >= 4





test['ad_revenue_per_retention_day'] = (test['total_ad_revenue'] / test['total_retention_days'].replace(0, 1))
test['level_completion_rate'] = (test['total_levels_completed'] / test['total_retention_days'].replace(0, 1))
adrevenue_columns = [f'AdRevenueD{i}' for i in range(1, 16)]
test['total_ad_revenue'] = test[adrevenue_columns].sum(axis=1)

retention_columns = [f'RetentionD{i}' for i in range(1, 16)]
test['total_retention_days'] = test[retention_columns].sum(axis=1)
LevelAdvancedCountD_columns = [f'LevelAdvancedCountD{i}' for i in range(1, 16)]
test['total_levels_completed'] = test[LevelAdvancedCountD_columns].sum(axis=1)
retention_days = ['RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6', 'RetentionD7']
test['early_adopter_flag'] = test[retention_days].sum(axis=1) >= 4








In [ ]:
pd.set_option("display.max_rows()",None)
def grab_col_names(dataframe, cat_th, car_th):
    """
    Grabs the column names according to data type and unique value counts.

    Parameters:
    - dataframe: pandas DataFrame to evaluate.
    - cat_th: Threshold for the number of unique values to consider a numerical column as categorical.
    - car_th: Threshold for the number of unique values to consider a categorical column as cardinal.

    Returns:
    - cat_cols: List of categorical column names.
    - cat_but_car: List of categorical but cardinal column names.
    - num_but_cat: List of numerical columns considered as categorical due to low number of unique values.
    - num_cols: List of numerical column names.
    """


    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    # Categorical columns that are not considered cardinal
    cat_but_not_car = [col for col in cat_cols if col not in cat_but_car]


    # Display unique value counts for all columns
    uniquevalue = dataframe.nunique().sort_values(ascending=False)
    print(f"The unique values:\n{uniquevalue}")

    # Output column lists
    print(f"cat_cols: {cat_cols}")
    print(f'num_cols: {num_cols}')
    print(f'num_cols but cat: {num_but_cat}')
    print(f'cat_but_car: {cat_but_car}')
    print(f'cat_but_not_car: {cat_but_not_car}')

    # Return the column lists
    return cat_cols, cat_but_car, num_but_cat, num_cols, cat_but_not_car

# Test the function with both train and test datasets

print("\n")
cat_cols_testFeatures, cat_but_car_testFeatures, num_but_cat_testFeatures, num_cols_testFeatures, cat_but_not_car_testFeatures = grab_col_names(testFeaturescopy, 10, 30)
print("\n")

cat_cols_testUsers, cat_but_car_testUsers, num_but_cat_testUsers, num_cols_testUsers, cat_but_not_car_testUsers = grab_col_names(test, 10, 30)




In [ ]:
#çalıştır
def checknull(data,percentvalue):

    upperPercentValueColumnlist=[col for col in data.columns if data[col].isnull().mean()*100>percentvalue]
    #boş değeri %15'ten fazla olan sütunlar
    betweenPercentValueColumnlist=[col for col in data.columns if data[col].isnull().mean()*100<percentvalue and data[col].isnull().mean()*100>0]
    #boş değeri %0-15 arasında olan sütunlar
    zeroPercentnullValueColumnlist=[col for col in data.columns if data[col].isnull().mean()==0]
    # boş değeri 0 olan değerler
    null_percent=(data.isnull().mean()*100).sort_values(ascending=False)

    print(f"""
The null values percent data:

    {null_percent}
          
          
Columns with null values greater than {percentvalue}%: {upperPercentValueColumnlist} 

Columns with null values is 0% : {zeroPercentnullValueColumnlist}

Columns with null values between 0 - {percentvalue}%: {betweenPercentValueColumnlist} 

""")
    return upperPercentValueColumnlist,betweenPercentValueColumnlist,zeroPercentnullValueColumnlist,null_percent

upperPercentValueColumnlistTrainFeatures,betweenPercentValueColumnlistTrainFeatures,zeroPercentnullValueColumnlistTrainFeatures,null_percentTrainFeatures=checknull(trainFeaturescopy,45)
upperPercentValueColumnlistTestFeatures,betweenPercentValueColumnlistTestFeatures,zeroPercentnullValueColumnlistTestFeatures,null_percentTestFeatures=checknull(test,45)
upperPercentValueColumnlistTestUsers,betweenPercentValueColumnlistTestUsers,zeroPercentnullValueColumnlistTestUsers,null_percentTestUsers=checknull(testUserscopy,45)
upperPercentValueColumnlistTrainUsers,betweenPercentValueColumnlistTrainUsers,zeroPercentnullValueColumnlistTrainUsers,null_percentTrainUsers=checknull(trainUserscopy,45)





In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

liste=["AdRevenueD0","AdRevenueD1"]


for col in liste:
    print(col, outlier_thresholds(train, col))
    print(col, outlier_thresholds(test, col))

for col in liste:
    print(col, check_outlier(train, col))
    print(col, check_outlier(test, col))
for col in liste:
    col, replace_with_thresholds(train, col)
    col, replace_with_thresholds(test, col)




In [ ]:
# OrdinalEncoder'ı tanıml
oe = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)

# Tüm string kolonları bul
string_cols = test.select_dtypes(include=['object',"bool"]).columns

# Tüm string kolonları dolas
for col in string_cols:
    # Convert column to string type to handle any mixed data types
    test[col] = train[col].astype(str)
    train[col] = test[col].astype(str)

    # Fit OrdinalEncoder
    train[[col]] = oe.fit_transform(train[[col]])
    test[[col]] = oe.transform(test[[col]])
train.shape,test.shape

In [ ]:
y = train["TARGET"]
X = train.drop(columns=["TARGET"],axis=1)



# Repeated K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=24)
scores = []
test_preds = np.zeros(len(test))  # Assuming `test` is your test set

# Perform K-Fold Cross-Validation
for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create and train CatBoostRegressor model
    model = CatBoostRegressor(random_state=41,verbose=False)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate RMSE score
    score = mean_squared_error(y_test, y_pred,squared=False)  # RMSE
    scores.append(score)

    print(f"Fold {fold + 1}: RMSE = {score:.4f}")

    # Average test set predictions across folds
    test_preds += model.predict(test) / kf.n_splits

# Calculate average RMSE score
mean_score = np.mean(scores)
print(f"Average RMSE = {mean_score:.4f}")

In [ ]:
data=pd.DataFrame({"ID":ID,"TARGET":test_preds})
data.set_index("ID",inplace=True)
data.to_csv("submission6.csv")

In [ ]:
#Average RMSE = 1.6074 kötü sonuç